In [9]:
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(10)
driver.get('https://www.flipkart.com')

driver.find_element_by_xpath('//div[@class="_2QfC02"]/button').click() #TO close login popup.

In [2]:
c = driver.find_element_by_tag_name('input')
c.click()
c.send_keys('Iphone')
driver.find_element_by_tag_name('button').click()

In [3]:
page1_links = driver.find_elements_by_class_name('_1fQZEK')
page1_links = [i.get_attribute('href') for i in page1_links]
driver.find_element_by_link_text('NEXT').click()
time.sleep(5)
page2_links = driver.find_elements_by_class_name('_1fQZEK')
page2_links = [i.get_attribute('href') for i in page2_links]
page2_links
links = page1_links+page2_links
links = links[:40]
driver.quit()

In [13]:
def cleanser(no):
    return re.sub(r'[^0-9]','',no)



def dataset(links):
    d_set = []
    for l in links:
        h = {'user-agent':'Sindhu'}
        r = requests.get(l,headers=h)
        time.sleep(10)
        html = r.text
        data = BeautifulSoup(html,'html.parser')
        Name = data.find(class_='B_NuCI').string
        Price = int(cleanser(data.find(class_ = '_25b18c').div.string.split()[0]))
        Prod_Rating = float(data.find(class_='_3LWZlK').get_text())
        c = data.find(id='sellerName')
        No_of_Prod_Rating = int(cleanser(data.find(class_ = '_2_R_DZ').span.contents[0].string.split()[0]))
        No_of_Prod_Reviews = int(cleanser(data.find(class_ = '_2_R_DZ').span.contents[2].string.split()[0]))
        Seller_Rating = float(c.div.text)
        Seller_Name = c.span.span.text
        d_set.append([Name,Price,Prod_Rating,No_of_Prod_Rating,No_of_Prod_Reviews,Seller_Name,Seller_Rating])
    return d_set

In [15]:
d_set = dataset(links)

AttributeError: 'NoneType' object has no attribute 'div'

In [12]:
df = pd.DataFrame(d_set,columns=['Name','Price','Prod_Rating','No_of_Prod_Rating','No_of_Prod_Reviews','Seller_Name','Seller_Rating'])
df.head()

,Name,Price,Prod_Rating,No_of_Prod_Rating,No_of_Prod_Reviews,Seller_Name,Seller_Rating
0,"APPLE iPhone SE (White, 64 GB)",39900,4.5,117892,9894,SuperComNet,4.6
1,"APPLE iPhone SE (Black, 64 GB)",39900,4.5,117892,9894,SuperComNet,4.6
2,"APPLE iPhone 12 (Blue, 128 GB)",70900,4.6,13045,1146,SuperComNet,4.6
3,"APPLE iPhone 12 Mini (White, 64 GB)",59900,4.5,4025,338,SuperComNet,4.6
4,"APPLE iPhone 12 Mini (Green, 64 GB)",59900,4.5,4025,338,SuperComNet,4.6


In [14]:
df.Price.mean()

63127.475